In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import astropy.units as u
from astropy.time import Time
import pytz
import astropy
from astropy.coordinates import SkyCoord
from astroplan import Observer, FixedTarget, observability_table, Constraint
from astropy.coordinates import get_sun, get_body, get_moon
from astroplan import moon_illumination
from astroplan import *
from astroplan import download_IERS_A
download_IERS_A()

# Setup

In [3]:
# Set up the location of observatory
palomar = Observer.at_site('Palomar')

We get the dataset from https://www.cosmos.esa.int/web/hipparcos/sample-tables-1

In [4]:
data = pd.read_csv('data2.csv')

In [5]:
#Pick out the objects having names
mask = list()
for n in range(len(data['Name'])):
    if(isinstance(data['Name'][n],str)):
        mask.append(True)
    else:
        mask.append(False)
data[mask].to_csv('hipparcos.csv')

In [6]:
star_table = pd.read_csv('hipparcos.csv')

In [7]:
#Set the observing window
obs_dates = list()
for i in range(15,31):
    date = "2019-10-" + str(i)
    time = Time(date)
    obs_dates.append(time)

# Visibility

In [8]:
#Complie a list of all defined objects
stars = list()
for s in range(len(star_table['Name'])):
    coords = SkyCoord(star_table['ra (deg)'][s]*u.deg, 
                                star_table['dec (deg)'][s]*u.deg, 
                                frame='icrs')
    stars.append(FixedTarget(name=star_table['Name'][s], coord=coords))

#Create an observability table

In [9]:
time = Time(["2019-10-15 00:00", "2019-10-30 23:59"])

In [10]:
#The moon's illumination values are from: https://www.calendar-12.com/moon_calendar/2019/october
#Use the mean value of the moon's illumination during our observing window
constraint = [AltitudeConstraint(30*u.deg, 90*u.deg),AirmassConstraint(2), 
              AtNightConstraint.twilight_astronomical(),MoonIlluminationConstraint(0.4),
              MoonSeparationConstraint(min = 10*u.deg)]

In [11]:
observability_table(observer = palomar,
                    constraints = constraint,
                    targets = stars,
                    time_range = time)

target name,ever observable,always observable,fraction of time observable
str14,bool,bool,float64
alpha_Cen_C,False,False,0.0
alpha^2_Cen,False,False,0.0
alpha^1_Cen,False,False,0.0
Barnard's_star,True,False,0.0078125
alpha_CMa,True,False,0.040364583333333336
epsilon_Eri,True,False,0.11067708333333333
61_Cyg_A,True,False,0.07161458333333333
alpha_CMi,True,False,0.052083333333333336
61_Cyg_B,True,False,0.07161458333333333


In [12]:
#Pick ten observable objects for the following steps
targets = list()
targets.append(stars[4])
targets.append(stars[5])
targets.append(stars[6])
targets.append(stars[7])
targets.append(stars[8])
targets.append(stars[3])
targets.append(stars[38])
targets.append(stars[37])
targets.append(stars[36])
targets.append(stars[29])

In [13]:
#Check the visibility of the ten objects with the highest value of moon's illumination
constraint = [AltitudeConstraint(30*u.deg, 90*u.deg),AirmassConstraint(2), 
              AtNightConstraint.twilight_astronomical(),MoonIlluminationConstraint(0.95),
              MoonSeparationConstraint(min = 10*u.deg)]
observability_table(observer = palomar,
                    constraints = constraint,
                    targets = targets,
                    time_range = time)

target name,ever observable,always observable,fraction of time observable
str14,bool,bool,float64
alpha_CMa,True,False,0.00390625
epsilon_Eri,True,False,0.014322916666666666
61_Cyg_A,True,False,0.016927083333333332
alpha_CMi,True,False,0.005208333333333333
61_Cyg_B,True,False,0.016927083333333332
Barnard's_star,True,False,0.006510416666666667
delta_Eri,True,False,0.013020833333333334
gamma_Lep,True,False,0.005208333333333333
chi^1_Ori,True,False,0.01171875


In [14]:
#Check the visibility of the ten objects with the lowest value of moon's illumination
constraint = [AltitudeConstraint(30*u.deg, 90*u.deg),AirmassConstraint(2), 
              AtNightConstraint.twilight_astronomical(),MoonIlluminationConstraint(0.01),
              MoonSeparationConstraint(min = 10*u.deg)]
observability_table(observer = palomar,
                    constraints = constraint,
                    targets = targets,
                    time_range = time)

target name,ever observable,always observable,fraction of time observable
str14,bool,bool,float64
alpha_CMa,True,False,0.06510416666666667
epsilon_Eri,True,False,0.13411458333333334
61_Cyg_A,True,False,0.07161458333333333
alpha_CMi,True,False,0.07942708333333333
61_Cyg_B,True,False,0.07161458333333333
Barnard's_star,True,False,0.0078125
delta_Eri,True,False,0.13541666666666666
gamma_Lep,True,False,0.07291666666666667
chi^1_Ori,True,False,0.11588541666666667


Therefore, these ten objects can be observed through the whole observing window.

# Moon Phase

We found the moon phases from: https://www.calendar-12.com/moon_calendar/2019/october
And based on the moon phases calendar, the moon will be waning gibbous on Oct 15th, 2019 (the start of our observing window) and keep waning until Oct 27th, 2019. Then the moon will be waxing from a new one at the end of our window.

In [15]:
#The moon's illumination
for i in range(len(obs_dates)):
    moon_ratio = moon_illumination(obs_dates[i])
    print(obs_dates[i])
    print(moon_ratio)
    print("\n")

2019-10-15 00:00:00.000
0.9865561943920241


2019-10-16 00:00:00.000
0.9562520875685778


2019-10-17 00:00:00.000
0.9074664247952995


2019-10-18 00:00:00.000
0.841225405905627


2019-10-19 00:00:00.000
0.759277143961225


2019-10-20 00:00:00.000
0.6641422433892282


2019-10-21 00:00:00.000
0.5591863674409921


2019-10-22 00:00:00.000
0.4487097215990062


2019-10-23 00:00:00.000
0.33800752114318117


2019-10-24 00:00:00.000
0.23329781313288983


2019-10-25 00:00:00.000
0.14137422531697247


2019-10-26 00:00:00.000
0.06888788130022877


2019-10-27 00:00:00.000
0.021331664431187447


2019-10-28 00:00:00.000
0.0020219486781750673


2019-10-29 00:00:00.000
0.011482916033771717


2019-10-30 00:00:00.000
0.047509435596680194




Combined with the moon's illumination, the moon will interfere with our observation somewhat at the beginning of our observing window. And with the moon waning later, the interference would be ignored.

# Visibility one month later

In [16]:
#New time range
time2 = Time(["2019-11-15 00:00", "2019-11-30 23:59"])

In [17]:
obs_dates2 = list()
for i in range(15,31):
    date = "2019-11-" + str(i)
    time = Time(date)
    obs_dates2.append(time)

In [18]:
# The moon's illumination values are from: https://www.calendar-12.com/moon_calendar/2019/november
#Use the mean value of the moon's illumination during the observing window again
illum = list()
for i in range(len(obs_dates2)):
    moon_ratio = moon_illumination(obs_dates2[i])
    illum.append(moon_ratio)
np.mean(illum)

0.3486490005684685

In [19]:
constraint = [AltitudeConstraint(30*u.deg, 90*u.deg),AirmassConstraint(2), 
              AtNightConstraint.twilight_astronomical(),MoonIlluminationConstraint(0.35),
              MoonSeparationConstraint(min = 10*u.deg)]

In [20]:
observability_table(observer = palomar,
                    constraints = constraint,
                    targets = targets,
                    time_range = time2)

target name,ever observable,always observable,fraction of time observable
str14,bool,bool,float64
alpha_CMa,True,False,0.06640625
epsilon_Eri,True,False,0.07421875
61_Cyg_A,True,False,0.020833333333333332
alpha_CMi,True,False,0.07682291666666667
61_Cyg_B,True,False,0.020833333333333332
Barnard's_star,False,False,0.0
delta_Eri,True,False,0.078125
gamma_Lep,True,False,0.046875
chi^1_Ori,True,False,0.078125


Thus, the situation will be worse for the ten objects.